# Firm Name Matching: Crunchbase vs Pitchbook

This notebook matches firms from Crunchbase (`samplecrunchbase3.csv`) to Pitchbook (`samplepitchbook3.csv`) using the `name_matching` library.

## Approach
1. **Preprocessing**: Clean company names by removing special characters, collapsing whitespace
2. **Name Matching**: Use the `NameMatcher` library with multiple distance metrics:
   - `discounted_levenshtein`: Handles character-level edits with position weighting
   - `SSK`: String Subsequence Kernel for partial matches
   - `fuzzy_wuzzy_token_sort`: Token-based matching that handles word reordering
3. **Filtering**: Keep only high-confidence matches (score > 80)
4. **Export**: Save results to CSV


In [ ]:
# Import required libraries
import pandas as pd
import re
from name_matching.name_matcher import NameMatcher
from tqdm import tqdm

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)


In [ ]:
# Load the datasets
print("Loading Crunchbase data...")
df_cb = pd.read_csv("samplecrunchbase3.csv")
print(f"Crunchbase: {len(df_cb):,} companies")

print("\nLoading Pitchbook data...")
df_pb = pd.read_csv("samplepitchbook3.csv")
print(f"Pitchbook: {len(df_pb):,} companies")

print("\n--- Crunchbase Sample ---")
display(df_cb.head())

print("\n--- Pitchbook Sample ---")
display(df_pb.head())


In [ ]:
def preprocess_company_name(text):
    """
    Preprocess company names for matching:
    - Handle missing values
    - Replace non-alphanumeric characters with spaces
    - Collapse multiple spaces
    - Strip leading/trailing whitespace
    """
    if pd.isna(text):
        return ""
    # Replace non-letters/numbers with space
    text = re.sub(r"[^A-Za-z0-9]", " ", str(text))
    # Collapse multiple spaces into one and strip
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Test preprocessing
test_names = ["perplexity.ai", "something-ai", "facebook, inc.", "X.com", None]
print("Preprocessing examples:")
for name in test_names:
    print(f"  '{name}' -> '{preprocess_company_name(name)}'")


In [ ]:
# Apply preprocessing to both datasets
print("Preprocessing company names...")

# Crunchbase: company name is in 'co1' column
df_cb["co1_clean"] = df_cb["co1"].apply(preprocess_company_name)

# Pitchbook: company name is in 'co2' column
df_pb["co2_clean"] = df_pb["co2"].apply(preprocess_company_name)

print(f"\nCrunchbase cleaned names sample:")
print(df_cb[["co1", "co1_clean"]].head(10))

print(f"\nPitchbook cleaned names sample:")
print(df_pb[["co2", "co2_clean"]].head(10))


In [ ]:
# For initial testing, use a sample to verify the matching works
# The full dataset has 448K Crunchbase and 86K Pitchbook companies
# This would take a very long time, so we'll start with a sample

SAMPLE_SIZE_PB = 1000  # Number of Pitchbook companies to match
USE_FULL_CB = True     # Use full Crunchbase as master (recommended for accuracy)

# Sample Pitchbook data (to be matched)
df_pb_sample = df_pb.head(SAMPLE_SIZE_PB).copy()
print(f"Using {len(df_pb_sample):,} Pitchbook companies for matching")

# Use full Crunchbase as master data
if USE_FULL_CB:
    df_cb_master = df_cb.copy()
    print(f"Using all {len(df_cb_master):,} Crunchbase companies as master data")
else:
    df_cb_master = df_cb.head(10000).copy()
    print(f"Using {len(df_cb_master):,} Crunchbase companies as master data (sampled)")


In [ ]:
# Initialize the NameMatcher
# Parameters explanation:
# - top_n: Number of candidate matches to consider for each name
# - lowercase: Convert names to lowercase before matching
# - punctuations: Remove punctuation (False = don't remove, we already cleaned)
# - remove_ascii: Remove non-ASCII characters
# - legal_suffixes: Remove common legal suffixes (Inc, LLC, Corp, etc.)
# - common_words: Remove common words that don't help matching
# - verbose: Print progress messages

print("Initializing NameMatcher...")
matcher = NameMatcher(
    top_n=10,
    lowercase=True,
    punctuations=False,
    remove_ascii=True,
    legal_suffixes=True,
    common_words=True,
    verbose=True
)

# Set distance metrics for fuzzy matching
# These metrics handle different types of name variations:
# - discounted_levenshtein: Character edits with position weighting
# - SSK: String Subsequence Kernel for substring matching
# - fuzzy_wuzzy_token_sort: Handles word reordering ("ABC Inc" vs "Inc ABC")
matcher.set_distance_metrics([
    'discounted_levenshtein',
    'SSK',
    'fuzzy_wuzzy_token_sort'
])
print("Distance metrics set.")


In [ ]:
# Load Crunchbase as master data
# This builds the index that will be searched against
print("Loading master data (Crunchbase)...")
print("This may take a few minutes for large datasets...")

matcher.load_and_process_master_data(
    column='co1_clean',
    df_matching_data=df_cb_master,
    transform=True
)
print("Master data loaded!")


In [ ]:
# Perform the name matching
# This finds the best match in Crunchbase for each Pitchbook company
print(f"Matching {len(df_pb_sample):,} Pitchbook companies...")
print("This may take several minutes...")

matches = matcher.match_names(
    to_be_matched=df_pb_sample,
    column_matching='co2_clean'
)

print(f"\nMatching complete! Found {len(matches):,} matches.")
print("\nSample matches:")
display(matches.head(10))


In [ ]:
# Merge the match results with original data
# The matches dataframe contains:
# - original_name: The cleaned Pitchbook name
# - match_name: The matched Crunchbase name
# - score: Match confidence score (0-100)
# - match_index: Index in the Crunchbase dataframe

print("Merging results...")

# First, merge Pitchbook sample with match results
combined = pd.merge(
    df_pb_sample.reset_index(drop=True),
    matches,
    how='left',
    left_index=True,
    right_index=True
)

# Now bring in the Crunchbase data for matched companies
# We'll add the Crunchbase columns based on match_index
cb_cols_to_add = ['uuid', 'co1', 'legalname1', 'state1']
for col in cb_cols_to_add:
    combined[f'cb_{col}'] = combined['match_index'].apply(
        lambda idx: df_cb_master.iloc[int(idx)][col] if pd.notna(idx) and idx >= 0 else None
    )

print(f"Combined dataset: {len(combined):,} rows")
display(combined.head(10))


In [ ]:
# Analyze match score distribution
print("=== Match Score Distribution ===")
print(combined['score'].describe())

print("\n=== Score Buckets ===")
score_buckets = pd.cut(
    combined['score'],
    bins=[0, 50, 60, 70, 80, 90, 100],
    labels=['0-50', '50-60', '60-70', '70-80', '80-90', '90-100']
)
print(score_buckets.value_counts().sort_index())

# Visualize if matplotlib is available
try:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10, 5))
    combined['score'].hist(bins=50, ax=ax, edgecolor='black')
    ax.set_xlabel('Match Score')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Match Scores')
    ax.axvline(x=80, color='red', linestyle='--', label='Threshold (80)')
    ax.legend()
    plt.tight_layout()
    plt.show()
except ImportError:
    print("(matplotlib not available for visualization)")


In [ ]:
# Filter to high-confidence matches (score > 80)
SCORE_THRESHOLD = 80

high_confidence = combined[combined['score'] > SCORE_THRESHOLD].copy()
print(f"High-confidence matches (score > {SCORE_THRESHOLD}): {len(high_confidence):,}")
print(f"Match rate: {len(high_confidence) / len(combined) * 100:.1f}%")

print("\n=== Sample High-Confidence Matches ===")
display(high_confidence[['co2', 'cb_co1', 'score', 'state2', 'cb_state1']].head(20))


In [ ]:
# Check for exact matches (after cleaning)
combined['exact_match'] = combined['co2_clean'].str.lower() == combined['match_name'].str.lower()

exact_matches = combined[combined['exact_match']]
print(f"Exact matches (after cleaning): {len(exact_matches):,}")
print(f"Exact match rate: {len(exact_matches) / len(combined) * 100:.1f}%")

print("\n=== Sample Exact Matches ===")
display(exact_matches[['co2', 'cb_co1', 'score']].head(10))


In [ ]:
# Analyze state matching for high-confidence matches
# This helps validate match quality - same company should often be in same state

# Normalize state codes for comparison
high_confidence['state_match'] = (
    high_confidence['state2'].str.lower().str.strip() == 
    high_confidence['cb_state1'].str.lower().str.strip()
)

state_match_count = high_confidence['state_match'].sum()
print(f"High-confidence matches with same state: {state_match_count:,}")
print(f"State agreement rate: {state_match_count / len(high_confidence) * 100:.1f}%")

# Show some mismatches for manual review
state_mismatches = high_confidence[~high_confidence['state_match']]
print(f"\n=== State Mismatches (for review) ===")
display(state_mismatches[['co2', 'cb_co1', 'score', 'state2', 'cb_state1']].head(10))


In [ ]:
# Prepare final export dataset
# Select and rename columns for clarity

export_cols = [
    'idp',              # Pitchbook ID
    'co2',              # Pitchbook company name (original)
    'cb_uuid',          # Crunchbase UUID
    'cb_co1',           # Crunchbase company name (original)
    'cb_legalname1',    # Crunchbase legal name
    'score',            # Match score
    'exact_match',      # Whether names match exactly after cleaning
    'state2',           # Pitchbook state
    'cb_state1',        # Crunchbase state
]

# Filter columns that exist
export_cols = [c for c in export_cols if c in combined.columns]

export_df = combined[export_cols].copy()

# Rename for clarity
export_df = export_df.rename(columns={
    'idp': 'pitchbook_id',
    'co2': 'pitchbook_name',
    'cb_uuid': 'crunchbase_uuid',
    'cb_co1': 'crunchbase_name',
    'cb_legalname1': 'crunchbase_legal_name',
    'score': 'match_score',
    'state2': 'pitchbook_state',
    'cb_state1': 'crunchbase_state'
})

print("Export dataset prepared:")
display(export_df.head(10))


In [ ]:
# Export results to CSV
# Save all matches
all_matches_path = "firm_matches_all.csv"
export_df.to_csv(all_matches_path, index=False)
print(f"Saved all matches to: {all_matches_path}")

# Save high-confidence matches only
high_conf_path = "firm_matches_high_confidence.csv"
high_conf_export = export_df[export_df['match_score'] > SCORE_THRESHOLD]
high_conf_export.to_csv(high_conf_path, index=False)
print(f"Saved high-confidence matches to: {high_conf_path}")

print(f"\n=== Summary ===")
print(f"Total Pitchbook companies processed: {len(export_df):,}")
print(f"All matches saved: {len(export_df):,}")
print(f"High-confidence matches (score > {SCORE_THRESHOLD}): {len(high_conf_export):,}")


## Running on Full Dataset

To run on the full dataset:

1. Change `SAMPLE_SIZE_PB` to a larger number or set it to `len(df_pb)` 
2. Be aware this will take significantly longer:
   - 86K Pitchbook companies × 448K Crunchbase companies
   - Estimated time: Several hours
   
**Recommendation**: Run in batches or on a machine with more resources.


In [ ]:
# Configuration for full dataset run (uncomment to use)
# WARNING: This will take a long time!

# SAMPLE_SIZE_PB = len(df_pb)  # Use all Pitchbook companies
# USE_FULL_CB = True           # Use all Crunchbase companies
# 
# # Then re-run cells starting from "sample-for-testing"
